In [ ]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import fisher_exact
import shelve
import pickle
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
import csv
from dfply import *
import itertools
import os
import logging
from sys import getsizeof
import sklearn
import time
from sklearn.metrics import roc_auc_score
from catboost import Pool, cv
import xgboost
import catboost
import scipy.stats as st

import importlib
from joblib import Parallel, delayed
from joblib import parallel_backend

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
#from statsmodels.tsa.vector_ar.var_model import VARY
from statsmodels.tsa.stattools import adfuller, kpss, grangercausalitytests, pacf
from statsmodels.graphics.tsaplots import plot_pacf
import warnings
warnings.filterwarnings("ignore")
import scipy
import datetime

from joblib import Parallel, delayed

import scipy
import importlib
from itertools import combinations
import forestplot as fp
import statsmodels.api as sm
import pyarrow
import concurrent.futures

In [ ]:
# 2021
# site='KUMC'
# print(site)
# import myits
# importlib.reload(myits)
# datafolder = '/home/hoyinchan/blue/Data/data2021raw/'
# myitsX = myits.myits()
# myitsX.get_onsetdata(site=site)                
# myitsX.get_data(site=site)
# myitsX.get_meddata(site=site)
# myitsX.set_loincs_pair()
# myitsX.med_code_transform()
# myitsX.extract_med_data()
# myitsX.lab = None
# with open(datafolder+'myits2_tmp_'+site+'.pkl', "wb") as f:
#      pickle.dump(myitsX,f)
# myitsX.run()

In [ ]:
site = 'UTHSCSA'
'/home/hoyinchan/blue/Data/data2022/'+site+'/p0_onset_'+site+'.parquet'

In [ ]:
#sites = ['UTHSCSA', 'UTSW', 'MCW', 'UofU', 'UIOWA', 'UMHC', 'UNMC', 'KUMC', 'UPITT']
sites = ['UIOWA', 'UMHC', 'UNMC', 'KUMC', 'UPITT']
#sites = ['UTHSCSA']
for site in sites:
    try:
        print(site)
        import myits
        importlib.reload(myits)
        datafolder = '/home/hoyinchan/blue/Data/data2022/'

        myitsX = myits.myits()
        myitsX.get_onsetdata(site=site)                
        myitsX.get_data(site=site)
        myitsX.get_meddata(site=site)
        myitsX.set_loincs_pair()
        myitsX.med_code_transform()
        myitsX.extract_med_data()
        myitsX.lab = None
        with open(datafolder+'myits2_tmp_'+site+'.pkl', "wb") as f:
             pickle.dump(myitsX,f)
        print(f"Finished {site}")
        # myitsX.run()
    except:
        pass

# myitsX.combine_output(prefix=datafolder+'myco4_corr_', outname='mlr7_cofounder')
# myitsX.combine_output(prefix=datafolder+'myco4_corrsig_', outname='mlr7sig_cofounder')
# myitsX.combine_output(prefix=datafolder+'myco4_corrraw_', outname='mlr7raw_cofounder')

# myitsX.calculate_all3(mydatatype='slope')
# myitsX.calculate_all3(mydatatype='intercept')

In [ ]:
sites = ['UTHSCSA', 'UTSW', 'MCW', 'UofU', 'UIOWA', 'UMHC', 'UNMC', 'KUMC', 'UPITT']

In [ ]:
import myits
importlib.reload(myits)
datafolder = '/home/hoyinchan/blue/Data/data2022/'

#for site in sites:
def get_labxx(site):    
    try:
        myitsX = myits.myits()
        with open(datafolder+'myits2_tmp_'+site+'.pkl', "rb") as f:
            myitsX2 = pickle.load(f)
        myitsX.copy(myitsX2)
        myitsX.calculate_pretreatment()
        myitsX.pre_post_count()
        myitsX.labxx['site'] = site
        return myitsX.labxx
    except Exception as e: 
        print('not working for site:', site, 'Error:', e)
        return None

In [ ]:
def parallel_processing(sites):
    labXs = []
    with concurrent.futures.ProcessPoolExecutor(max_workers=11) as executor:
        results = executor.map(get_labxx, sites)        
#        for result in results:
#            labXs.extend(result)
    return results

In [ ]:
labXs = parallel_processing(sites)

labX2 = list(labXs)
labX3 = [x for x in labX2 if isinstance(x, pd.DataFrame)]
labX4 = pd.concat(labX3)

In [ ]:
labX4['PATID'] = labX4['PATID'].astype(str)
labX4['ENCOUNTERID'] = labX4['ENCOUNTERID'].astype(str)
labX4['ONSETS_ENCOUNTERID'] = labX4['ONSETS_ENCOUNTERID'].astype(str)
labX4['LAB_RESULT_CM_ID'] = labX4['LAB_RESULT_CM_ID'].astype(str)
labX4['ID'] = labX4['PATID']+'_'+labX4['ENCOUNTERID']+'_'+labX4['site']
labX4['DAYS_SINCE_ADMIT'] = labX4['DAYS_SINCE_ADMIT'].astype(int)

In [ ]:
labX4['MEDADMIN_CODE_NEW'] = 'MED:'+labX4['MEDADMIN_TYPE']+':'+labX4['MEDADMIN_CODE']

In [ ]:
labX4.to_pickle(datafolder+'allmedlabxx.pkl')

In [ ]:
labX40 = labX4.copy()
#labX40['RESULT_DATE'] = pd.to_datetime(labX40['RESULT_DATE'])
labX40 = labX40.drop(['LAB_ORDER_DATE', 'SPECIMEN_TIME', 'RESULT_DATE', 'RESULT_TIME', 'NORM_RANGE_LOW', 'NORM_RANGE_HIGH']+[x for x in labX4.columns if 'RAW' in x],axis=1)
labX40.to_parquet(datafolder+'allmedlabxx.parquet')

In [ ]:
dict_med =   {'MED:ATC:A07DA': 'DiphenoxylateLoperamide(LK):A07DA',
              'MED:ATC:C03CA': 'FurosemideBumetanideTorsemide(LKLNa):C03CA',
              'MED:ATC:C03CB': 'FurosemideBumetanideTorsemide(LKLNa):C03CA',
              'MED:ATC:C03EB': 'FurosemideBumetanideTorsemide(LKLNa):C03EB',
              'MED:ATC:C03AA': 'hydrochlorothiazide(LNa):C03AA',
              'MED:ATC:C03AB': 'hydrochlorothiazide(LNa):C03AB',
              'MED:ATC:C03AH': 'chlorothiazide(LNa):C03AH',
              'MED:ATC:C03BA': 'chlorthalidone(LNa):C03BA',
              'MED:ATC:C03EA': 'hydrochlorothiazide(LNa):C03EA',
              'MED:ATC:C09DX': 'SacubitrilValsartan(HK):C09DX',
              'MED:ATC:C03DA': 'eplerenone(HK):C03DA',
              'MED:ATC:C03XA': 'tolvaptan(LNa):C03XA',
              'MED:ATC:C09XA': 'hydrochlorothiazide(LNa):C09XA',
              'MED:ATC:C09DX': 'hydrochlorothiazide(LNa):C09DX',
              'MED:ATC:C03AX': 'hydrochlorothiazide(LNa):C03AX',
              'MED:ATC:C09BX': 'hydrochlorothiazide(LNa):C09BX',
              'MED:ATC:M05BA': 'PamidronateZoledronate(HCa):M05BA',
              'MED:ATC:M05BB': 'etidronate(HCa):M05BB'}

In [ ]:
# lab_loinc = {'2823-3':'potassium',
#             '17861-6':'calcium',
#             '2951-2':'sodium', 
#             '2160-0':'sCr'}
lab_loinc = {'LG49936-4':'potassium',
                'LG49864-8':'calcium',
                'LG11363-5':'sodium', 
                'LG6657-3':'sCr'}

In [ ]:
for loinc in list(lab_loinc.keys()):
    for med in list(dict_med.keys()):
        try:
            nhanes2 = labX4.copy()

            nhanes2['posttreatment'] = ~nhanes2['pretreatment']
            nhanes2['posttreatment'] = nhanes2['posttreatment'].astype(int)

            nhanes2 = nhanes2[nhanes2['MEDADMIN_CODE_NEW'] == med]
            nhanes2 = nhanes2[nhanes2['LAB_LOINC'] == loinc]
            
            #nhanes2 = nhanes2[['ENCOUNTERID', 'RESULT_NUM', 'pretreatment2', 'SPECIMEN_DATE_TIME_DELTA']]
            #nhanes2['SPECIMEN_DATE_TIME_DELTA2'] = nhanes2['SPECIMEN_DATE_TIME_DELTA'].dt.total_seconds()/60/60

            nhanes2 = nhanes2[['ID', 'RESULT_NUM', 'posttreatment', 'DAYS_SINCE_ADMIT', 'site']]

            nhanes2_min = nhanes2[nhanes2['posttreatment']==1][['ID','DAYS_SINCE_ADMIT']].groupby(['ID']).min().reset_index()
            nhanes2_min.columns = ['ID','DAYS_SINCE_ADMIT_MIN']
            nhanes2 = nhanes2.merge(nhanes2_min, on = 'ID', how='inner') 
            
            nhanes2['DAYS_SINCE_ADMIT'] = nhanes2['DAYS_SINCE_ADMIT'] - nhanes2['DAYS_SINCE_ADMIT_MIN']
            nhanes2 = nhanes2.drop('DAYS_SINCE_ADMIT_MIN',axis=1)
            
            nhanes2.ID = pd.factorize(nhanes2['ID'])[0] + 1            
            print("Saving " + datafolder+''+'xtitsa_'+loinc.replace("-",'')+med.replace(":",'')+'.parquet')
            if nhanes2.shape[0] == 0:
                print(f"No element for {loinc}:{med}")
            else:
                nhanes2.to_parquet(datafolder+''+'xtitsa_'+loinc.replace("-",'')+med.replace(":",'')+'.parquet')
        except Exception as e: 
            print('not working for site:', loinc, med, 'Error:', e)

In [ ]:
# onsetsdfs = list()
# sites = ['UTHSCSA', 'KUMC', 'MCW', 'UMHC', 'UNMC', 'UTSW', 'UofU', 'UPITT', 'IUR', 'UIOWA']
# for site in sites:
#     onsetsdf = pd.read_csv(datafolder+site+'/raw/AKI_ONSETS.csv')
#     onsetsdf['site'] = site
#     onsetsdfs.append(onsetsdf)
# onsetsdfs = pd.concat(onsetsdfs)
# onsetsdfs['PATID'] = onsetsdfs['PATID'].astype(str)
# onsetsdfs['ENCOUNTERID'] = onsetsdfs['ENCOUNTERID'].astype(str)

# onsetsdfs =  onsetsdfs[['PATID','ENCOUNTERID', 'ADMIT_DATE', 'AKI1_ONSET', 'AKI2_ONSET', 'AKI3_ONSET']]
# onsetsdfs = onsetsdfs.bfill(axis=1)
# onsetsdfs = onsetsdfs[['PATID','ENCOUNTERID', 'ADMIT_DATE', 'AKI1_ONSET']]
# onsetsdfs['AKI1_ONSET'] = pd.to_datetime(onsetsdfs['AKI1_ONSET'])        

# onsetsdfs

In [ ]:
# #Senity Check

# med = 'MED:ATC:A07DA'
# loinc = '2160-0'

# datafolder = '/home/hoyinchan/blue/Data/data2021raw/'
# labX4 = pd.read_parquet(datafolder+'allmedlabxx.parquet')

# nhanes2 = labX4.copy()

# nhanes2.columns

# nhanes2['onset_since_med'] = (pd.to_datetime(nhanes2['AKI1_ONSET'])-pd.to_datetime(nhanes2['MEDADMIN_START_DATE_TIME'])).dt.days
# nhanes2 = nhanes2[nhanes2['onset_since_med']<=7]

# df = nhanes2.copy()
# # Filter dataframe for rows where 'posttreatment' is False and count by 'ENCOUNTERID'
# false_counts = df[df['posttreatment'] == False].groupby('ENCOUNTERID').size()

# # Filter dataframe for rows where 'posttreatment' is True and count by 'ENCOUNTERID'
# true_counts = df[df['posttreatment'] == True].groupby('ENCOUNTERID').size()

# # Find 'ENCOUNTERID' values that meet the criteria
# ids = false_counts[(false_counts > 7) & (true_counts > 5)].index

# print(ids.tolist())

# nhanes2['posttreatment'] = ~nhanes2['pretreatment']
# nhanes2['posttreatment'] = nhanes2['posttreatment'].astype(int)

# nhanes2 = nhanes2[nhanes2['MEDADMIN_CODE_NEW'] == med]
# nhanes2 = nhanes2[nhanes2['LAB_LOINC'] == loinc]

# nhanes2X = nhanes2[nhanes2['ENCOUNTERID'] == '140000055697415']

# nhanes2Xp = nhanes2X[['DAYS_SINCE_ADMIT', 'RESULT_NUM', 'pretreatment']].groupby(['DAYS_SINCE_ADMIT', 'pretreatment']).mean().reset_index()

# colors = nhanes2Xp['pretreatment'].map({True: 'red', False: 'blue'})

# plt.scatter(nhanes2Xp['DAYS_SINCE_ADMIT'], nhanes2Xp['RESULT_NUM'], c=colors)

# nhanes2.columns

# nhanes2[nhanes2['ENCOUNTERID'] == '140000055697415']['onset_since_med']

# #onsets =  pd.read_csv(datafolder+'KUMC'+'/raw/AKI_ONSETS.csv') 

# onsets.columns

# onsets[onsets['ENCOUNTERID']==927634][['SERUM_CREAT_BASE', 'AKI1_SINCE_ADMIT', 'AKI1_SCR','AKI1_INC']]

# #labs =  pd.read_csv(datafolder+'KUMC'+'/raw/AKI_LAB.csv') 

# labsX = labs[labs['ENCOUNTERID']==927634]

# labsX[labsX['LAB_LOINC']=='2160-0'].sort_values('DAYS_SINCE_ADMIT')

In [ ]:
# dict_med =   {'MED:ATC:A07DA': 'DiphenoxylateLoperamide(LK):A07DA',
#               'MED:ATC:C03CA': 'FurosemideBumetanideTorsemide(LKLNa):C03CA',
#               'MED:ATC:C03CB': 'FurosemideBumetanideTorsemide(LKLNa):C03CA',
#               'MED:ATC:C03EB': 'FurosemideBumetanideTorsemide(LKLNa):C03EB',
#               'MED:ATC:C03AA': 'hydrochlorothiazide(LNa):C03AA',
#               'MED:ATC:C03AB': 'hydrochlorothiazide(LNa):C03AB',
#               'MED:ATC:C03AH': 'chlorothiazide(LNa):C03AH',
#               'MED:ATC:C03BA': 'chlorthalidone(LNa):C03BA',
#               'MED:ATC:C03EA': 'hydrochlorothiazide(LNa):C03EA',
#               'MED:ATC:C09DX': 'SacubitrilValsartan(HK):C09DX',
#               'MED:ATC:C03DA': 'eplerenone(HK):C03DA',
#               'MED:ATC:C03XA': 'tolvaptan(LNa):C03XA',
#               'MED:ATC:C09XA': 'hydrochlorothiazide(LNa):C09XA',
#               'MED:ATC:C09DX': 'hydrochlorothiazide(LNa):C09DX',
#               'MED:ATC:C03AX': 'hydrochlorothiazide(LNa):C03AX',
#               'MED:ATC:C09BX': 'hydrochlorothiazide(LNa):C09BX',
#               'MED:ATC:M05BA': 'PamidronateZoledronate(HCa):M05BA',
#               'MED:ATC:M05BB': 'etidronate(HCa):M05BB'}

In [ ]:
# labX4= pd.read_pickle(datafolder+'allmedlabxx.pkl')

In [ ]:
# labX4['MEDADMIN_CODE_NEW'].unique()